In [ ]:
# Basic Imports
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Data Upload and Cleaning 



In [ ]:
# Upload training data 
train_features = pd.read_csv('features_train.tsv', sep = '\t')
train_labels = train_features.pop('DIFF')
BC_train = pd.read_csv('tfbs_score_BC217_train.tsv', sep = '\t')
YJ_train = pd.read_csv('tfbs_score_YJF153_train.tsv', sep = '\t' )

In [ ]:
# Upload testing data
test_features = pd.read_csv('features_test.tsv', sep = '\t')
BC_test = pd.read_csv('tfbs_score_BC217_test.tsv', sep = '\t')
YJ_test = pd.read_csv('tfbs_score_YJF153_test.tsv', sep = '\t' )

In [ ]:
# Looking at some info 
for column in train_features.columns :
  unique_values = train_features[column].unique()
  length = len(unique_values)

  if length < 20: 
    print(column + ' : ', unique_values , '\n')
  else:
    print(column + ' : ', length , '\n')


In [ ]:
# Function to clean the _features dataframes
def clean_features(df):
  # Drop columns with non relevant or little info
  df.drop('type', axis =1 , inplace = True) 
  df.drop('gene', axis =1 , inplace = True)
  df.drop(['len_intron','pos_intron','len_5UTRintron', 'pos_5UTRintron'], axis =1 , inplace = True)

  # Map chromosome numbers to numbers
  df['chromosome'] = df['chromosome'].map({'chr1':1,  'chr2':2, 'chr3':3, 'chr4':4, 'chr5':5, 
                                  'chr6':6, 'chr7':7, 'chr8':8, 'chr9':9,  'chr10':10,
                                  'chr11':11, 'chr12':12, 'chr13':13, 'chr14':14,
                                  'chr15':15, 'chr16':16 })
  return df 


In [ ]:
# Clean dataframes
train_features = clean_features(train_features)

# Also drop diff for test data since it contains no info
test_features.drop('DIFF', axis = 1, inplace=True)
test_features = clean_features(test_features)

Encode non numerical data

In [ ]:
# Figure out some info about the genotype data 
genotype_data = ['YJF153_geno_SNP_promoter', 'BC217_geno_SNP_promoter',
                 'YJF153_geno_MIXED_promoter','BC217_geno_MIXED_promoter',
                 'YJF153_geno_SNP_3end','BC217_geno_SNP_3end','YJF153_geno_MIXED_3end',
                 'BC217_geno_MIXED_3end', 'YJF153_geno_SNPs_gene','BC217_geno_SNPs_gene',
                 'YJF153_geno_MIXED_gene','BC217_geno_MIXED_gene']

largest = 0
running_total = 0 
running_num = 0 

for column in genotype_data:
  for i in range(train_features.shape[0]):
    try:
      current = len(train_features[column][i].split(':'))
      running_total = running_total + current 
      running_num =  running_num + 1
    except: continue
    if (largest == 0) | (current > largest) :
      largest = current
    
print('largest:' , largest)
print('total:', running_total, 'num:',running_num)
print('avg:' ,running_total/running_num)

largest: 277
total: 94818 num: 16824
avg: 5.635877318116976


In [ ]:
# Alternative Functions required for translating with new method
def checkIfNewChunk(existingChunks, chunk):
    if chunk in existingChunks:
        return False
    else:
        return True

def extractSequenceChunks(train_features, genotype_data):
    unique_seq_chunks = ['A','T','C','G']
    for column in genotype_data:
        for i in range(train_features.shape[0]):
            try:
                getChunks = train_features[column][i].split(":")
                for chunk in getChunks:
                    newChunk = checkIfNewChunk(unique_seq_chunks, chunk)
                    if newChunk:
                        unique_seq_chunks.append(chunk)
            except: continue
    np.save('uniqueSeqChunks', unique_seq_chunks)
    print('numUniqueSeq:', len(unique_seq_chunks))

def encodeGenotypes(value, uniqueSeqChunks):
  try:
      return uniqueSeqChunks.index(value)
  except:
      return len(uniqueSeqChunks)

def translateColumnChunks(train_features, genotype_data, uniqueSeqChunks ,cutoffSize=20):
    rows = train_features.shape[0]
    # Go through each column
    for column in genotype_data:
        # Allocate space for each translation of 20
        translatedColumn = np.zeros((rows, cutoffSize))
        for i in range(rows):
            if type(train_features[column][i]) != str:
                getChunks = [0]
            else:
                getChunks = train_features[column][i].split(":")
            for j in range(cutoffSize):
                try:
                    translatedColumn[i][j] = encodeGenotypes(getChunks[j], uniqueSeqChunks)+1
                except:
                    translatedColumn[i][j] = 0
        train_features.drop(columns=column, inplace=True)
        for i in range(cutoffSize):
            columnAdding = column + "_SplitPart_" + str(i)
            train_features[columnAdding] = translatedColumn[:,i].tolist()
    return train_features

# Method for translating the numeric input
def translateDistChunks(train_features, numerical_lists, cutoffSize=20):
    rows = train_features.shape[0]
    # Go through each column
    for column in numerical_lists:
        # Allocate space for each translation of 20
        translatedColumn = np.zeros((rows, cutoffSize), dtype=np.float32)
        for i in range(rows):
            if type(train_features[column][i]) != str:
                getChunks = [0]
            else:
                getChunks = train_features[column][i].split(":")
            for j in range(cutoffSize):
                try:
                    translatedColumn[i][j] = np.float32(getChunks[j])
                except:
                    translatedColumn[i][j] = 0
        train_features.drop(columns=column, inplace=True)
        for i in range(cutoffSize):
            columnAdding = column + "_SplitPart_" + str(i)
            train_features[columnAdding] = translatedColumn[:,i].tolist()
    return train_features

# Requirements
genotype_data = ['YJF153_geno_SNP_promoter', 'BC217_geno_SNP_promoter',
                 'YJF153_geno_MIXED_promoter','BC217_geno_MIXED_promoter',
                 'YJF153_geno_SNP_3end','BC217_geno_SNP_3end','YJF153_geno_MIXED_3end',
                 'BC217_geno_MIXED_3end', 'YJF153_geno_SNPs_gene','BC217_geno_SNPs_gene',
                 'YJF153_geno_MIXED_gene','BC217_geno_MIXED_gene']
numerical_lists = ['dist_SNP_promoter', 'dist_MIXED_promoter','dist_SNPs_gene', 'dist_MIXED_gene']

# Code to execute
extractSequenceChunks(train_features,genotype_data)
uniqueSeqChunks = np.load("uniqueSeqChunks.npy")

train_features_translated = translateColumnChunks(train_features, genotype_data, uniqueSeqChunks)
train_features_translate = translateDistChunks(train_features_translated, numerical_lists)

test_features_translated = translateColumnChunks(test_features, genotype_data, uniqueSeqChunks)
test_features_translate = translateDistChunks(test_features_translated, numerical_lists)

numUniqueSeq: 866


In [ ]:
# So we don't need to change variable names later
train_features = train_features_translate.copy()
test_features = test_features_translate.copy()

In [ ]:
# Function to encode ACTG values in genotype data
def amino_to_num(string): 
  if type(string) == int:
    return string

  try:
    letters = string.split(':')
  except: 
    letters = ''

  letter_vocab = {'A': 1, 'T': 2, 'C': 3, 'G':4}
  numbers = ''

  # Encode 
  for letter in letters:
    if len(letter) == 1:
      number = str(letter_vocab[letter])
      numbers = numbers + number 
    elif len(letter) > 1:
      for char in letter:
       number = str(letter_vocab[char])
       numbers = numbers + number 
    numbers = numbers + '5'

  # Pad to 50 characters 
  num_to_pad = 50 - len(numbers)
  if num_to_pad >= 0:
    numbers = numbers + '0' * num_to_pad
  elif num_to_pad < 0:
    numbers = numbers[0:50]

  # Convert to integer
  numbers = int(numbers)

  return numbers


In [ ]:
# Encode the genotype data in training and testing dataframes 
#for column in genotype_data:
  #train_features[column]=train_features[column].map(amino_to_num)
  #test_features[column]=test_features[column].map(amino_to_num)

More data cleaning...

In [ ]:
# Function to get rid of colons in distance lists 
numerical_lists = ['dist_SNP_promoter', 'dist_MIXED_promoter','dist_SNPs_gene', 'dist_MIXED_gene']

In [ ]:

def remove_colons(string):
   if type(string) != str:
    return string

   try:
      letters = string.split(':')
   except: 
     letters = ''
  
   list_nums = ''
   numbers=string.split(':')
   for number in numbers[0:50]:
      list_nums = list_nums + number
   
   num=int(list_nums)

   return num



In [ ]:
# Get rid of colons in distance lists
#for column in numerical_lists:
  #train_features[column]=train_features[column].map(remove_colons)
  #test_features[column] = train_features[column].map(remove_colons)

In [ ]:

# Split the column with dashed values and adds them to the dataSet at the end could make this more robust for scenario of n_exons but did this to starts
train_features[["pos_exon1_start", "pos_exon1_end", "pos_exon2_start", "pos_exon2_end", "pos_exon3_start", "pos_exon3_end"]] = train_features["pos_exon"].str.split(r'\W', expand = True)
train_features.drop(columns='pos_exon', inplace=True)

# Replace all Nan and Na values with 0
train_features.fillna(0, inplace=True)

# Convert all columns that are possible into floats and print to screen a list where not possible
failFloat32 = []
for column in train_features.columns:
    try: 
        train_features[column]=train_features[column].astype(np.float64)
    except:
        failFloat32.append(column)
        
print(failFloat32)
  

['SGD']


In [ ]:
# Split the column with dashed values and adds them to the dataSet at the end could make this more robust for scenario of n_exons but did this to starts
splitColumns = test_features["pos_exon"].str.split(r'\W', expand = True)
try:
  test_features[["pos_exon1_start", "pos_exon1_end", "pos_exon2_start", "pos_exon2_end", "pos_exon3_start", "pos_exon3_end"]] = splitColumns
except:
  columns = ["pos_exon1_start", "pos_exon1_end", "pos_exon2_start", "pos_exon2_end", "pos_exon3_start", "pos_exon3_end"]
  numRows, numColumns = splitColumns.shape
  # Grab the max range
  for i in range(6):
    if i < numColumns:
      columnToAdd = splitColumns.iloc[:,i]
    else:
      columnToAdd = np.zeros((numRows,1))
    test_features[columns[i]]= columnToAdd
    

test_features.drop(columns='pos_exon', inplace=True)

# Replace all Nan and Na values with 0
test_features.fillna(0, inplace=True)

# Convert all columns that are possible into floats and print to screen a list where not possible
failFloat32 = []
for column in test_features.columns:
  try: 
        test_features[column]=test_features[column].astype(np.float64)
  except:
        failFloat32.append(column)
        
print(failFloat32)

['SGD']


Normalize numerical columns up to this point

In [ ]:
# Normalization
train_SGD = train_features.pop('SGD')
test_SGD = test_features.pop('SGD')
train_features=(train_features-train_features.min())/(train_features.max()-train_features.min())
test_features= (test_features-test_features.min())/(test_features.max()-test_features.min())

train_features['SGD']=train_SGD
test_features['SGD']=test_SGD

Join features dataframe with TFBS dataframes

In [ ]:
# Function to prepare the TFBS data 
def prepare_TFBS_data(df):
    df.rename({'gene':'SGD'},axis=1, inplace=True)
    df.drop(df.columns[1], axis =1 , inplace = True)
    df.set_index('SGD', inplace=True)

    return df

In [ ]:
# Prepare TFBS data
# Training
YJ_train = prepare_TFBS_data(YJ_train)
BC_train = prepare_TFBS_data(BC_train)
TFBS_train = abs(YJ_train-BC_train)

# Testing
YJ_test = prepare_TFBS_data(YJ_test)
BC_test = prepare_TFBS_data(BC_test)
TFBS_test = abs(YJ_test-BC_test)


In [ ]:
# Join Data 
# Training
train_features = train_features.join(TFBS_train, on='SGD')

# Testing 
test_features = test_features.join(TFBS_test, on='SGD')

In [ ]:
# Make copies so up to now is preserved 
training_features = train_features.copy()
testing_features = test_features.copy()

# Get rid of SGD for training data 
training_features.fillna(0, inplace=True)
training_features.drop('SGD', axis = 1, inplace=True)

# Pop SGD gene identifier for testing data 
testing_features.fillna(0, inplace=True)
test_SGD = testing_features.pop('SGD')


In [ ]:
for column in testing_features.columns:
  if testing_features[column].isnull().any()==True:
    print(column)

NOTES FROM DIANA 8/24 WORK

After using a difference between the TFBS data rather than each table separately, it seems that the # epochs can be increased (to 100 at least) without causing the convergence on a value we were seeing before. So thats good :)

Additional Notes:

- We could potentially encode the genotype SNPs using a one-hot encoding - https://www.nature.com/articles/s41467-020-19921-4

- Does standardising the strand type make sense: AKA writing a script to convert all to strand 0 or all to strand 1. Since they are just inverse of one another

- Data in TFBS seems to be structured where each group of 3 columns is one feature with data score,position and strand: Information on the TFBS, https://pubmed.ncbi.nlm.nih.gov/12176838/


Creating the model 

In [ ]:
# Parameters for training
epochs = 100
batch_size = 15 
validation_split = 0.2 
learning_rate = 0.01

In [ ]:
# General Model Framework
# Create Model
model= tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, activation = 'relu', input_shape=(training_features.shape[1],)))
model.add(tf.keras.layers.Dense(128, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(1))

# Compile Model 
model.compile(optimizer='adam', loss= 'mean_squared_error' ,metrics= 'mse')

In [ ]:
# Train Model
model.fit(training_features, train_labels, epochs = epochs, batch_size = batch_size, validation_split= validation_split)

Epoch 1/100
149/149 [==============================] - 2s 10ms/step - loss: 0.6022 - mse: 0.6022 - val_loss: 0.4750 - val_mse: 0.4750
Epoch 2/100
149/149 [==============================] - 1s 8ms/step - loss: 0.5504 - mse: 0.5504 - val_loss: 0.4432 - val_mse: 0.4432
Epoch 3/100
149/149 [==============================] - 1s 8ms/step - loss: 0.5317 - mse: 0.5317 - val_loss: 0.4459 - val_mse: 0.4459
Epoch 4/100
149/149 [==============================] - 1s 8ms/step - loss: 0.5267 - mse: 0.5267 - val_loss: 0.4565 - val_mse: 0.4565
Epoch 5/100
149/149 [==============================] - 1s 9ms/step - loss: 0.5283 - mse: 0.5283 - val_loss: 0.4406 - val_mse: 0.4406
Epoch 6/100
149/149 [==============================] - 1s 9ms/step - loss: 0.5191 - mse: 0.5191 - val_loss: 0.4669 - val_mse: 0.4669
Epoch 7/100
149/149 [==============================] - 1s 8ms/step - loss: 0.5172 - mse: 0.5172 - val_loss: 0.4550 - val_mse: 0.4550
Epoch 8/100
149/149 [==============================] - 1s 8ms/step -

In [ ]:
# Test of accuracy on training data 
predictions = model.predict(training_features)
for p in range(10,15):
   prediction = str(predictions[p,0])
   actual = train_labels[p]
   print(prediction, actual)

0.82352847 0.299317082414784
0.48318005 0.151013990079708
0.684715 0.974245527905857
0.6879412 0.557997957647867
2.4197829 1.90212685959492


In [ ]:
# Predict using model
predictions = model.predict(testing_features)

# Write a CSV file 
div_predictions=predictions.flatten()
div_df=pd.DataFrame(data=div_predictions, columns = ['DIFF'])
div_df['SGD']=test_SGD
div_df.set_index('DIFF', inplace=True)

div_df.to_csv('prediction.csv')

div_df

,SGD
DIFF,
1.509571,YOL031C
0.652272,YER032W
1.076889,YPL172C
1.600312,YJL155C
0.789321,YOL093W
...,...
1.788472,YDL198C
1.720836,YOR317W
0.736998,YDR236C


In [ ]:
# This box gives some stats on the predictions 
div=div_df.reset_index()
div['DIFF'].describe()

count    698.000000
mean       0.966464
std        0.680192
min        0.308674
25%        0.602034
50%        0.747374
75%        1.027082
max        6.707862
Name: DIFF, dtype: float64

In [ ]:
train_labels.describe()

count    2793.000000
mean        0.924559
std         0.734205
min         0.000000
25%         0.426961
50%         0.711980
75%         1.195470
max         5.293015
Name: DIFF, dtype: float64